## ANARX-Linearisierung

Vorraussetzung:

Sei $\mathbf{x}(t) = \begin{bmatrix}x_1(t), & \dots & ,x_n(t)\end{bmatrix}$ und $\mathbf{u}(t) = \begin{bmatrix}u_1,(t) & \dots & ,u_n(t)\end{bmatrix}$

Torch ANARX-Modell mit $n$ Subnetzen $N_n$ zusammengefasst als $\mathbf{N}(\mathbf{x}, \mathbf{u})=\begin{bmatrix}N_1(x_1, \mathbf{u}) \\ \vdots \\ N_{n-1}(x_{n-1}, \mathbf{u}) \\ N_n(\mathbf{u})\end{bmatrix}$

Sei $U_n = \delta_{i+1,j}$ die obere Verschiebungsmatrix, dann gilt für die Zustandsübergangsfunktion:

$\mathbf{x}(t+1) = U_n \cdot \mathbf{x}(t) + N\left(\mathbf{x}(t), \mathbf{u}\left(t+1\right)\right)$

bzw.:

$F(\mathbf{x},\mathbf{u}) = U_n \cdot \mathbf{x} + N(\mathbf{x}, \mathbf{u})$

Dann kann die Linearisierung von $F$ im Punkt $(\mathbf{x}_0, \mathbf{u_0}) $ wie folgt bestimmt werden:

$F_L(\mathbf{x}_0+\Delta \mathbf{x}, \mathbf{u}_0+ \Delta \mathbf{u})=\left.\dfrac{\partial F}{\partial \mathbf{x},\mathbf{u}}\right|_{(\mathbf{x}_0, \mathbf{u_0})} \cdot \Delta  \mathbf{x} + \left.\dfrac{\partial F}{\partial \mathbf{x},\mathbf{u}}\right|_{(\mathbf{x}_0, \mathbf{u_0})} \cdot \Delta  \mathbf{u} + F(\mathbf{x}, \mathbf{u})$

In [19]:
import numpy as np
A = np.array([1,2,3])
B = np.zeros([3,3])
B[0,1] = 1
B[1,2] = 1
np.matmul(B,A)

array([2., 3., 0.])

In [11]:
import torch
from ANARX import LAGNET

# Beispieldaten
x = torch.tensor([2.], requires_grad=True)
u = torch.tensor([3.], requires_grad=True)

# Hier wird als Beispiel ein Subnetz aus der ANARX Klasse erzeugt
# Der Code unten funktioniert allerdings für beliebige torch.Tensor-wertige Funktionen
N = LAGNET(2, n_hidden=2, layersize=3, afunc=torch.tanh, bias=True)

result = N(torch.cat((x,u))) # Auswertung von N
result.backward() # Beim Backward-Pass bestimmt Autograd die Gradienten

# Die Gradienten werden in der .grad-Property der Tensoren gespeichert
print(x.grad)
print(u.grad)

# Wenn nur die Jacobimatrix benötigt wird kann sie mit diesem Befehl direkt bestimmt werden
J = torch.autograd.functional.jacobian(N, torch.cat((x,u)))

# Die Ergebnisse der Methoden stimmen überein
assert torch.equal(J.squeeze(), torch.cat((x.grad,u.grad)))




tensor([0.0059])
tensor([0.0168])
